In [ ]:
import nbformat

notebook_path = "RAG_Techniques/test.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


In [1]:
!pip install langchain-community langchain-text-splitters  pypdf  langchain-huggingface sentence-transformers faiss-cpu ollama langchain-ollama   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 82.4 MB/s eta 0:00:00


In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 39.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.3.7 which is incompatible.


In [27]:
import os
from google.colab import drive

# 1. 掛載 Google Drive
drive.mount('/content/drive')

# 2. 設定模型儲存路徑 (建議在 Drive 建立一個專門的資料夾)
# 這裡設定為 MyDrive 下的 RAG_project/ollama_models 資料夾
my_model_folder = '/content/drive/MyDrive/RAG/ollama_models'

# 如果資料夾不存在，建立它
if not os.path.exists(my_model_folder):
    os.makedirs(my_model_folder)

# 3. 【關鍵】設定環境變數，讓 Ollama 知道去哪裡找模型
os.environ['OLLAMA_MODELS'] = my_model_folder

print(f"Ollama 模型路徑已設定為: {os.environ['OLLAMA_MODELS']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ollama 模型路徑已設定為: /content/drive/MyDrive/RAG/ollama_models


In [28]:
import os
import subprocess
import time
from google.colab import drive


# --- 2. 修正並安裝 Ollama ---
print("正在安裝相依套件 zstd...")
!sudo apt-get install -y zstd  # <--- 新增這一行解決你的錯誤

print("正在安裝 Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 3. 啟動 Ollama 服務 ---
print("正在啟動 Ollama 服務...")
# 使用完整路徑以防萬一 (通常是 /usr/local/bin/ollama)
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 等待服務啟動
print("等待服務啟動中 (約 10 秒)...")
time.sleep(10)

# --- 4. 測試連線 ---
try:
    # 檢查服務是否活著
    check = subprocess.run(["curl", "-s", "http://localhost:11434"], capture_output=True, text=True)
    if "Ollama is running" in check.stdout:
        print("✅ 成功：Ollama 服務已在背景執行！")
    else:
        print("⚠️ 警告：服務似乎未回應，請檢查日誌。")
except Exception as e:
    print(f"檢查連線時發生錯誤: {e}")

正在安裝相依套件 zstd...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
正在安裝 Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
正在啟動 Ollama 服務...
等待服務啟動中 (約 10 秒)...
✅ 成功：Ollama 服務已在背景執行！


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from typing import List,Dict,Any,Tuple
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from pydantic import Field,BaseModel
from langchain_core.prompts import PromptTemplate
import faiss
from langchain_ollama import ChatOllama
from langchain_core.retrievers import BaseRetriever

In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [7]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
chunk_size=512
chunk_overlap=64
query="What the CRAG proposed to improve "
llm=ChatOllama(model="qwen3:4b-instruct-2507-q8_0",temperature=0)

In [10]:
def replace_tab_with_space(docs):
    for doc in docs:
        doc.page_content=doc.page_content.replace("\t"," ")
    return docs

def encode_pdf(path,chunk_size,chunk_overlap):

    loader=PyPDFLoader(path)
    documents=loader.load()

    splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunks=splitter.split_documents(documents)

    cleaned_chunks=replace_tab_with_space(chunks)

    vectorstore=FAISS.from_documents(cleaned_chunks,embedding_model)

    return vectorstore

vectorstore=encode_pdf(path,chunk_size,chunk_overlap)

##Method 1: LLM based function to rerank the retrieved documents

In [36]:
class RatingScore(BaseModel):
    relevace_score:float = Field(...,description="The relevance score of documents to a query")

def rerank_documents(query:str, docs:List[Document], top_n:int=3 )->List[Document]:
    template="""
    on the scale of 1-10,rate the relevace of following document to the query.
    consider the specific context and intent of query,not just keyword matches.
    Query:{query}
    Document:{doc}
    relevace score:
    """
    prompt=PromptTemplate.from_template(
        template=template,
        )

    llm_chain=prompt | llm.with_structured_output(RatingScore)

    scored_docs=[]
    for doc in docs:
        input_data={"query":query,"doc":doc.page_content}
        score=llm_chain.invoke(input_data).relevace_score

        try:
            score=float(score)
        except ValueError:
            score=0
        scored_docs.append((doc,score))

    reranked_docs=sorted(scored_docs,key= lambda x : x[1], reverse=True)
    return [doc for doc,_ in reranked_docs[:top_n]]


In [37]:
initial_docs = vectorstore.similarity_search(query, k=15)
print(initial_docs)

[Document(id='283e9bea-12dd-4cc4-b0fd-95763ef21175', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-08T01:19:39+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-08T01:19:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf', 'total_pages': 16, 'page': 14, 'page_label': '15'}, page_content='improvement of our method.\nSelf-CRAG: To demonstrate that our plug-and-\nplay approach can be utilized in other concurrent\nstudies, we specifically designed to insert our\nCRAG into the Self-RAG (Asai et al., 2024)\nframework and named it Self-CRAG. Self-RAG'), Document(id='6483e03b-d484-459c-afa2-210ff95957a4', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-08T01:19:39+00:00', 'author': '', 'keywords': '', 'modd

In [42]:
!ps aux | grep ollama

root       13175  0.1  0.2 1785252 33436 ?       Sl   12:56   0:00 ollama serve
root       13303  0.0  0.0   7376  3484 ?        S    12:56   0:00 /bin/bash -c ps aux | grep ollama
root       13305  0.0  0.0   6484  2404 ?        S    12:56   0:00 grep ollama


In [43]:
reranked_docs = rerank_documents(query, initial_docs)

In [44]:
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:2]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs[:2]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print first 200 characters of each document

Top initial documents:

Document 1:
improvement of our method.
Self-CRAG: To demonstrate that our plug-and-
play approach can be utilized in other concurrent
studies, we specifically designed to insert our
CRAG into the Self-RAG (Asai et al., 2024)
framework and named it Self-CRAG. Self-RAG

Document 2:
Generation ( CRAG ) is proposed to self-correct
the results of retriever and improve the utilization
of documents for augmenting generation. A
lightweight retrieval evaluator is designed to
assess the overall quality of retrieved documents
for a query. This serves as a crucial component
in RAG, contributing to informative generation
by reviewing and evaluating the relevance
and reliability of the retrieved documents. A
confidence degree is quantified based on which
Query: What the CRAG proposed to improve 

Top reranked documents:

Document 1:
where the retriever returns inaccurate results and,
to the best of our knowledge, makes the first
attempt to design corrective strategies for RAG

##Method 2: Cross Encoder models

In [8]:
model_name="BAAI/bge-reranker-v2-m3"
reranker=CrossEncoder(model_name)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/393 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [16]:
class CrossEncoderRetriever(BaseRetriever, BaseModel):
  vectorstore:Any = Field(..., description="The vectorstore for initial retrieval")
  reranker:Any = Field(..., description="model for rreranking")
  k:int = Field(default=5, description="Number of documents to retrieve initially")
  rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

  #這是一個 告訴 Pydantic 放鬆型別檢查的開關,讓上面的vectorstore:Any不屬於python的內建型別
  class Config:
    arbitrary_types_allowed = True

  def _get_relevant_documents(self, query:str) -> List[Document]:
    initial_docs = self.vectorstore.similarity_search(query, k=self.k)
    pair=[[query,doc.page_content] for doc in initial_docs]
    scores=self.reranker.predict(pair)

    scored_docs=sorted(zip(initial_docs,scores),key=lambda x:x[1],reverse=True)
    return [doc for doc,_ in scored_docs[:self.rerank_top_k]]

  async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")


/tmp/ipython-input-1206998908.py:1: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class CrossEncoderRetriever(BaseRetriever, BaseModel):


In [18]:
# Create the cross-encoder retriever
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    reranker=reranker,
    k=10,  # Retrieve 10 documents initially
    rerank_top_k=5  # Return top 5 after reranking
)

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
template="""
Answer the question base on content
question:{question}
context:{context}
"""
prompt=ChatPromptTemplate.from_template(template)

rag_chain = (
    {
        "context": cross_encoder_retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)



In [31]:
!ps aux | grep ollama

root        6653  0.5  0.3 1859496 50656 ?       Sl   14:04   0:00 ollama serve
root        6761  8.9  6.2 48197524 837024 ?     Sl   14:04   0:09 /usr/local/bin/ollama runner --ollama-engine --model /content/drive/MyDrive/RAG/ollama_models/blobs/sha256-af6e43ab13611311226e6f809f6a39b1a87b6df613bbf79468059f92ce819c4a --port 44439
root        7213  0.0  0.0   7376  3456 ?        S    14:06   0:00 /bin/bash -c ps aux | grep ollama
root        7215  0.0  0.0   6484  2596 ?        S    14:06   0:00 grep ollama


In [30]:
rag_chain.invoke(query)

'CRAG proposed to improve the **ability of automatic self-correction** and the **efficient utilization of retrieved documents** in Retrieval-Augmented Generation (RAG) systems. Specifically, it addresses the issue of inaccurate retrieval results by introducing a lightweight retrieval evaluator that assesses the quality of retrieved documents. Based on this evaluation, a confidence degree is quantified to trigger three knowledge retrieval actions, thereby correcting flawed retrieval outcomes and ensuring only relevant and reliable information is used for generation. This enhances both the robustness and performance of RAG, especially in short- and long-form generation tasks.'